In [ ]:
%load_ext autoreload

%autoreload 2
import matplotlib.pyplot as plt
import numpy as np
from skimage.exposure import rescale_intensity

from sentinelhub import CRS, BBox, DataCollection, SHConfig
from sentinelhub import SentinelHubCatalog
from sentinelhub import SHConfig
from oxeo.core.stac import landsat
from oxeo.core.stac.constants import USWEST_URL ,ELEMENT84_URL
uswest_config = SHConfig()
uswest_config.sh_base_url = USWEST_URL

uswest_catalog = SentinelHubCatalog(config=uswest_config)
eu_catalog = SentinelHubCatalog(SHConfig())
from oxeo.core.data import get_aoi_from_landsat_shub_catalog, get_aoi_from_stac_catalog
import os
os.environ["AWS_REQUEST_PAYER"] = "requester"

from oxeo.core.utils import get_bounding_box

In [ ]:
box = get_bounding_box({"geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              14.911966323852539,
              37.30573714593416
            ],
            [
              14.987583160400392,
              37.30573714593416
            ],
            [
              14.987583160400392,
              37.345050859282736
            ],
            [
              14.911966323852539,
              37.345050859282736
            ],
            [
              14.911966323852539,
              37.30573714593416
            ]
          ]
        ]
      }})

In [ ]:
bbox = BBox(box, crs=CRS.WGS84)#BBox([49.9604, 44.7176, 51.0481, 45.2324], crs=CRS.WGS84) #BBox((-71.40254974365233, -46.9537775782648, -71.26213073730467, -46.89163931213445), crs=CRS.WGS84)#
time_interval = "2020-12-10", "2021-02-01" #"1994-12-10", "1995-02-01"
search_params = {
}


In [ ]:
s1_aoi = get_aoi_from_stac_catalog(catalog=eu_catalog,
                                   data_collection=DataCollection.SENTINEL1,
                                   bbox=bbox,
                                   time_interval=time_interval,
                                   search_params={})


                                        


In [ ]:
s1_aoi


In [ ]:
vv = s1_aoi.sel(band="vv").mean(axis=0).compute()
vh = s1_aoi.sel(band="vh").mean(axis=0).compute()


In [ ]:
vv_thresh = 80
rgb = np.where(
    vv < vv_thresh,
    [vv, 8 * vv, 0.5 + 3 * vv + 2000 * vh],
    [3 * vv, 1.1 * vv + 8.75 * vh, 1.75 * vh],
).transpose(1, 2, 0)

cutoff = 2000
dis = rescale_intensity(rgb, in_range=(0, cutoff), out_range=(0, 1))

plt.imshow(dis)

In [ ]:
s2_aoi = get_aoi_from_stac_catalog(catalog=ELEMENT84_URL,
                                   data_collection="sentinel-s2-l2a-cogs",
                                   bbox=bbox,
                                   time_interval=time_interval,
                                   search_params={})

img = s2_aoi.sel(band=["B04","B03","B02"])[:5].mean(axis=0)[:,:,:].values.transpose(1,2,0)
plt.imshow(rescale_intensity(img,(np.nanmin(img), np.nanmax(img))))

In [ ]:
landsat_aoi = get_aoi_from_stac_catalog(catalog=uswest_catalog,
                                        data_collection=DataCollection.LANDSAT_OT_L2,
                                        bbox=bbox,
                                        time_interval=time_interval,
                                        search_params={},
                                        category="T1")
                                        

img = landsat_aoi.sel(band=['SR_B4', 'SR_B3', 'SR_B2'])[:5].mean(axis=0)[:,:,:].values.transpose(1,2,0)
plt.imshow(rescale_intensity(img,(np.nanmin(img), np.nanmax(img))))


In [ ]:
landsat_aoi = get_aoi_from_stac_catalog(catalog=uswest_catalog,
                                        data_collection=DataCollection.LANDSAT_ETM_L2,
                                        bbox=bbox,
                                        time_interval=time_interval,
                                        search_params={},
                                        category="T1")
                                        

img = landsat_aoi.sel(band=['red', 'green', 'blue'])[:5].median(axis=0)[:,:,:].values.transpose(1,2,0)
plt.imshow(rescale_intensity(img,(np.nanmin(img), np.nanmax(img))))


In [ ]:
landsat_aoi = get_aoi_from_stac_catalog(catalog=uswest_catalog,
                                        data_collection=DataCollection.LANDSAT_TM_L2,
                                        bbox=bbox,
                                        time_interval= ("1994-12-10", "1995-02-01"),
                                        search_params={},
                                        category="T1")
                                        

img = landsat_aoi.sel(band=['red', 'green', 'blue'])[:5].median(axis=0)[:,:,:].values.transpose(1,2,0)
plt.imshow(rescale_intensity(img,(np.nanmin(img), np.nanmax(img))))


# Prediction

In [ ]:

import numpy as np


from oxeo.water.models.segmentation import Segmentation2DPredictor
from oxeo.core.models.tile import load_tile_from_stac_as_dict, load_aoi_from_stac_as_dict, tile_from_id, TilePath, tile_to_geom
from oxeo.core import data
import matplotlib.pyplot as plt

In [ ]:
from oxeo.core.constants import BAND_PREDICTOR_ORDER



In [ ]:
s2_predictor = Segmentation2DPredictor(ckpt_path="../data/semseg_epoch_012.ckpt", fs=None,
                                    bands=BAND_PREDICTOR_ORDER["sentinel-2"])
preds = s2_predictor.predict_stac_aoi(constellation="sentinel-2",
                           catalog=ELEMENT84_URL,
                           data_collection="sentinel-s2-l2a-cogs",
                           bbox=bbox,
                           time_interval=time_interval,
                           search_params={},
                           revisit=slice(0,2))
                           
plt.imshow(preds[0])

In [ ]:
l8_predictor = Segmentation2DPredictor(ckpt_path="../data/semseg_epoch_012.ckpt", fs=None,
                                    bands=BAND_PREDICTOR_ORDER["landsat-8"],
                                    target_resolution=30)

preds = l8_predictor.predict_stac_aoi(constellation="landsat-8",
                           catalog=uswest_catalog,
                           data_collection=DataCollection.LANDSAT_OT_L2,
                           bbox=bbox,
                           time_interval=time_interval,
                           search_params={},
                           revisit=slice(0,2))
plt.imshow(preds[0])

In [ ]:
l7_predictor = Segmentation2DPredictor(ckpt_path="../data/semseg_epoch_012.ckpt", fs=None,
                                    bands=BAND_PREDICTOR_ORDER["landsat-7"],
                                    target_resolution=30)

preds = l7_predictor.predict_stac_aoi(constellation="landsat-7",
                           catalog=uswest_catalog,
                           data_collection=DataCollection.LANDSAT_ETM_L2,
                           bbox=bbox,
                           time_interval=time_interval,
                           search_params={},
                           revisit=slice(0,2))
plt.imshow(preds[0])

In [ ]:
l5_predictor = Segmentation2DPredictor(ckpt_path="../data/semseg_epoch_012.ckpt", fs=None,
                                    bands=BAND_PREDICTOR_ORDER["landsat-5"],
                                    target_resolution=60)

preds = l5_predictor.predict_stac_aoi(constellation="landsat-5",
                           catalog=uswest_catalog,
                           data_collection=DataCollection.LANDSAT_TM_L2,
                           bbox=bbox,
                           time_interval=("1994-12-10", "1995-02-01"),
                           search_params={},
                           revisit=slice(0,2))
plt.imshow(preds[0])